# AutoML using H2O Ai

Hello Dockshippers !
In this notebook , i'll be showing a baseline model from H2O ai's automl. 

Here's an excellent documentation on H2O ai's automl.

https://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html

 It's still in development for multiclass classification AFAIK, but works like charm for regression and binary classification.

 I'll be demonstrating the simple and easy way of training using H2O ai

In [1]:
import h2o
from h2o.automl import H2OAutoML

In [2]:
h2o.init(
    nthreads=-1,     # number of threads when launching a new H2O server
    max_mem_size=12  # in gigabytes

)


Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.8" 2020-07-14; OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpbz49kt63
  JVM stdout: /tmp/tmpbz49kt63/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpbz49kt63/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.1
H2O_cluster_version_age:,8 months and 5 days !!!
H2O_cluster_name:,H2O_from_python_unknownUser_fg2yg3
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [3]:
import pandas as pd
ss = pd.read_csv('./segmind-dockship/dataset/sample_submission.csv')
test_features = pd.read_csv('./segmind-dockship/dataset/TEST.csv')
train_features = pd.read_csv('./segmind-dockship/dataset/TRAIN.csv')
store = pd.read_csv("./segmind-dockship/dataset/store.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
train = pd.merge(train_features,store,on="Store")
test = pd.merge(test_features,store,on="Store")

In [5]:
train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
train

Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
1,5,2015-07-31 00:00:00,5263,555,1,1,,1,c,a,1270,9,2008,0,nan,nan,nan
1,4,2015-07-30 00:00:00,5020,546,1,1,,1,c,a,1270,9,2008,0,nan,nan,nan
1,3,2015-07-29 00:00:00,4782,523,1,1,,1,c,a,1270,9,2008,0,nan,nan,nan
1,2,2015-07-28 00:00:00,5011,560,1,1,,1,c,a,1270,9,2008,0,nan,nan,nan
1,1,2015-07-27 00:00:00,6102,612,1,1,,1,c,a,1270,9,2008,0,nan,nan,nan
1,7,2015-07-26 00:00:00,0,0,0,0,,0,c,a,1270,9,2008,0,nan,nan,nan
1,6,2015-07-25 00:00:00,4364,500,1,0,,0,c,a,1270,9,2008,0,nan,nan,nan
1,5,2015-07-24 00:00:00,3706,459,1,0,,0,c,a,1270,9,2008,0,nan,nan,nan
1,4,2015-07-23 00:00:00,3769,503,1,0,,0,c,a,1270,9,2008,0,nan,nan,nan
1,3,2015-07-22 00:00:00,3464,463,1,0,,0,c,a,1270,9,2008,0,nan,nan,nan


In [7]:
x = train.columns[1:]
y = "Sales"
x.remove(y)

In [8]:
aml = H2OAutoML(max_models=2, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [9]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20210416_092103,131038,361.992,131038,236.383,nan
XGBoost_1_AutoML_20210416_092103,141224,375.797,141224,247.356,nan
XGBoost_2_AutoML_20210416_092103,173718,416.795,173718,272.086,nan


Now this is a pretty good score considering we had to do zero or minimal feature engineering  :)